# People Counter Sample Application

This notebook shows how to create an object detection app for Panorama using a pretrained MXNet model.

By completing this notebook, you will learn:
* How to write a Python script for your app that takes in camera streams, performs inference, and outputs results
* How to test your code using the Panorama emulator running inside this Jupyter notebook, which saves you deployment time
* How to use an MXNet object detection model with your app
* How to programmatically package and deploy applications using the Panorama CLI
* How to use an abstract camera node and over ride the camera programatically

--- 

1. [Prerequisites](#Prerequisites)
1. [Set up](#Set-up)
1. [Import model](#Import-model)
1. [Write and test app code](#Write-and-test-app-code-in-notebook)
1. [Package app](#Package-app)
1. [Deploy app to device](#Deploy-app-to-device)

# Prerequisites

1. In a terminal session on this Jupyter notebook server, run `aws configure`. This allows this notebook server to access Panorama resources and deploy applications on your behalf.

# Set up

Import libraries for use with this notebook environment, you do not need these libraries when you write your application code.

In [ ]:
import sys
import os
import time
import json

import boto3
import sagemaker

import matplotlib.pyplot as plt
from IPython.core.magic import register_cell_magic

sys.path.insert( 0, os.path.abspath( "../../panorama_sdk" ) )
import utils

# instantiate boto3 clients
s3_client = boto3.client('s3')
panorama_client = boto3.client('panorama')

# configure matplotlib
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,20)

# register custom magic command
@register_cell_magic
def run_and_save(line, cell):
    'Run and save python code block to a file'
    with open(line, 'wt') as fd:
        fd.write(cell)
    code = compile(cell, line, 'exec')
    exec(code, globals())

## Notebook parameters
Global constants that help the notebook create Panorama resources on your behalf.

In [ ]:
# Device ID, should look like: device-oc66nax4cgzwhyuaeyifrqowue
DEVICE_ID = input( 'DEVICE_ID (format: device-*)' ).strip()

# Enter your S3 bucket info here
S3_BUCKET = input( 'S3_BUCKET' ).strip()

# Enter your desired AWS region
AWS_REGION = input( 'AWS_REGION (e.g. us-east-1)' ).strip()  

ML_MODEL_FNAME = 'ssd_512_resnet50_v1_voc'

In [ ]:
c = utils.Config(

    # application name
    app_name = 'people_counter_app',

    ## package names and node names
    code_package_name = 'PEOPLE_COUNTER_CODE',
    model_package_name = 'SSD_MODEL',
    camera_node_name = 'abstract_rtsp_media_source',

    # models (model node name : compiled model path without platform dependent suffic)
    models = {
        "model_node" : "./models/" + ML_MODEL_FNAME,
    },

    # video file path to simulate camera stream
    videoname = '../../panorama_sdk/videos/TownCentreXVID.avi',

    # AWS account ID
    account_id = boto3.client("sts").get_caller_identity()["Account"],
)

utils.configure(c)

## Set up application

Every application uses the creator's AWS Account ID as the prefix to uniquely identifies the application resources. Running `panorama-cli import-application` replaces the generic account Id with your account Id.

In [ ]:
!cd ./people_counter_app && panorama-cli import-application

# Import model

We need to compile and import the model twice. Once for testing with this notebook server and once for deploying to the Panorama device.

While working with the Panorama sample code, we provide pretrained models for you to use. Locally, models are stored in `panorama_sdk/models`. This step downloads the model artifacts from our Amazon S3 bucket to the local folder. If you want to use your own models, put your tar.gz file into the `panorama_sdk/models folder`.

### Prepare model for testing with notebook server

In [ ]:
# Downloads pretrained model for this sample
utils.download_sample_model( ML_MODEL_FNAME, "./models" )

In [ ]:
# Upload the model to S3, compile it with SageMaker, download the result, and extract it
utils.prepare_model_for_test(
    region = AWS_REGION,
    data_shape = '{"data":[1,3,512,512]}',
    framework = 'MXNET',
    local_model_filepath = "./models/ssd_512_resnet50_v1_voc.tar.gz",
    s3_model_location = f"s3://{S3_BUCKET}/{c.app_name}/",
    compile_job_role = utils.resolve_sm_role(),
)

### Prepare model for deploying to Panorama device

In [ ]:
asset_name = 'model_asset'
package_path = f'packages/{c.account_id}-{c.model_package_name}-1.0'
descriptor_path = f'packages/{c.account_id}-{c.model_package_name}-1.0/descriptor.json'

In [ ]:
!cd ./people_counter_app && panorama-cli add-raw-model \
    --model-asset-name {asset_name} \
    --model-s3-uri s3://{S3_BUCKET}/{c.app_name}/{ML_MODEL_FNAME}.tar.gz \
    --descriptor-path {descriptor_path}  \
    --packages-path {package_path}

# Write and test app code in notebook

Every app has an entry point script, written in Python that pulls the frames from camera streams, performs inference, and send the results to the desired location. This file can be found in `your_app/packages/code_node/src/main.py`. Below, you will iterate on the code from within the notebook environment. The entry point file will be updated everytime you run the next notebook cell thanks to the `%%run_and_save people_counter_main.py`. This is a utility function to update the contents of the entry point script. 

The next cell will loop through the test video until you select Stop on your Jupyter notebook cell. 

### Iterating on Code Changes

To iterate on the code:
1. Make changes in the next cell. 
2. Stop the Notebook and Reset the Kernel.
3. Run the 3 Setup cells again before re-running the code cell below.

**CHANGE VIDEO** : For you to change video, please place the video in samples/panorama_sdk/videos and update the global variables with the video and extension name (video.avi for example)

Run only the Set up and Notebook Parameters cells before running this for reiteration

In [ ]:
%%run_and_save app.py

import json
import logging
import time
from logging.handlers import RotatingFileHandler

import boto3
import cv2
import numpy as np
import panoramasdk

class Application(panoramasdk.node):
    def __init__(self):
        """Initializes the application's attributes with parameters from the interface, and default values."""
        
        self.MODEL_NODE = "model_node"
        self.MODEL_DIM = 512
        self.frame_num = 0
        self.threshold = 50.
        # Desired class
        self.classids = [14.]
        
        try:
            # Parameters
            logger.info('Getting parameters')
            self.threshold = self.inputs.threshold.get()
        except:
            logger.exception('Error during initialization.')
        finally:
            logger.info('Initialiation complete.')
            logger.info('Threshold: {}'.format(self.threshold))
            

    def process_streams(self):
        """Processes one frame of video from one or more video streams."""
        self.frame_num += 1
        logger.debug(self.frame_num)

        # Loop through attached video streams
        streams = self.inputs.video_in.get()
        for stream in streams:
            self.process_media(stream)

        self.outputs.video_out.put(streams)

    def process_media(self, stream):
        """Runs inference on a frame of video."""
        image_data = preprocess(stream.image, self.MODEL_DIM)
        logger.debug(image_data.shape)

        # Run inference
        inference_results = self.call({"data":image_data}, self.MODEL_NODE)

        # Process results (object deteciton)
        self.process_results(inference_results, stream)

    def process_results(self, inference_results, stream):
        """Processes output tensors from a computer vision model and annotates a video frame."""
        if inference_results is None:
            logger.warning("Inference results are None.")
            return

        num_people = 0

        class_data = None # Class Data
        bbox_data = None # Bounding Box Data
        conf_data = None # Confidence Data
        
        # Pulls data from the class holding the results
        # inference_results is a class, which can be iterated through
        # but inference_results has no index accessors (cannot do inference_results[0])

        k = 0
        for det_data in inference_results:
            if k == 0:
                class_data = det_data[0]
            if k == 1:
                conf_data = det_data[0]
            if k == 2:
                bbox_data = det_data[0]
                for a in range(len(conf_data)):
                    if conf_data[a][0] * 100 > self.threshold and class_data[a][0] in self.classids:
                        (left, top, right, bottom) = np.clip(det_data[0][a]/self.MODEL_DIM,0,1)
                        stream.add_rect(left, top, right, bottom)
                        num_people += 1
                    else:
                        continue
            k += 1
        
        logger.info('# people {}'.format(str(num_people)))
        stream.add_label('# people {}'.format(str(num_people)), 0.1, 0.1)


def preprocess(img, size):
    """Resizes and normalizes a frame of video."""
    resized = cv2.resize(img, (size, size))
    mean = [0.485, 0.456, 0.406]  # RGB
    std = [0.229, 0.224, 0.225]  # RGB
    img = resized.astype(np.float32) / 255.  # converting array of ints to floats
    r, g, b = cv2.split(img) 
    # normalizing per channel data:
    r = (r - mean[0]) / std[0]
    g = (g - mean[1]) / std[1]
    b = (b - mean[2]) / std[2]
    # putting the 3 channels back together:
    x1 = [[[], [], []]]
    x1[0][0] = r
    x1[0][1] = g
    x1[0][2] = b
    return np.asarray(x1)

def get_logger(name=__name__,level=logging.INFO):
    logger = logging.getLogger(name)
    logger.setLevel(level)
    handler = RotatingFileHandler("/opt/aws/panorama/logs/app.log", maxBytes=100000000, backupCount=2)
    formatter = logging.Formatter(fmt='%(asctime)s %(levelname)-8s %(message)s',
                                    datefmt='%Y-%m-%d %H:%M:%S')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    return logger

def main():
    try:
        logger.info("INITIALIZING APPLICATION")
        app = Application()
        logger.info("PROCESSING STREAMS")
        while True:
            app.process_streams()
    except:
        logger.exception('Exception during processing loop.')

logger = get_logger(level=logging.INFO)
main()

# Package app

Updates the app to be deployed with the recent code

In [ ]:
file_name = 'app.py'
package_path = f"./{c.app_name}/packages/{c.account_id}-{c.code_package_name}-1.0/src/{file_name}"
utils.update_descriptor( c.account_id, c.code_package_name, file_name )

os.system( f"mv {file_name} {package_path}" )

## Update camera streams

In the AWS Panorama console, you can select the camera streams, but programmatically, you need to define the camera stream info for the cameras you are using with the app.

We used an ```abstract data source``` here, usually this lets you select the pre-created camera source from the console. But programatically, we have to do the following steps


- Create Camera
- Create Override json file
- Include the Override json file while are deploying the application

### Create New Camera

Because we are using an ```abstract_rtsp_media_source```, we have to create a camera before we can use the ```abstract_rtsp_media_source```

**NOTE** : Update your RTSP Info in the next cell, Username, Password and RTSP Stream URL

In [ ]:
CAMERA_NAME = "test_rtsp_camera"
CAMERA_CREDS = '{"Username":"root","Password":"Aws2017!","StreamUrl": "rtsp://192.168.0.201/onvif-media/media.amp?profile=profile_1_h264&sessiontimeout=60&streamtype=unicast"}'

In [ ]:
res = !aws panorama create-node-from-template-job --template-type RTSP_CAMERA_STREAM \
    --output-package-name {CAMERA_NAME} \
    --output-package-version '2.0' \
    --node-name {CAMERA_NAME} \
    --template-parameters '{CAMERA_CREDS}'

# FIXME : camera node creation fails if it already exists.
# Should either ignore the already-exist error, or delete the node at the end of this notebook

res = ''.join(res)
print(res)
res_json = json.loads(res)

In [ ]:
!aws panorama describe-node-from-template-job --job-id {res_json['JobId']}

## Overriding camera node

If you want to override the camera configuration at deployment (for ex. deploy to another site) you can provide a deployment time override. Go to `people_counter_app/deployment_overrides/override_camera.json` file and replace YOUR_AWS_ACCOUNT_ID with your ACCOUNT_ID and YOUR_CAMERA_NAME with your camera name.

In [ ]:
# Update Account ID
with open( f"./{c.app_name}/deployment_overrides/override_camera.json", "r" ) as fd:
    override_json = json.load(fd)

override_json['nodeGraphOverrides']['packages'][0]['name'] = '{}::{}'.format(c.account_id, CAMERA_NAME)
override_json['nodeGraphOverrides']['nodes'][0]['name'] = CAMERA_NAME
override_json['nodeGraphOverrides']['nodes'][0]['interface'] = '{}::{}.{}'.format(c.account_id, CAMERA_NAME, CAMERA_NAME)    
override_json['nodeGraphOverrides']['nodeOverrides'][0]['with'][0]['name'] = CAMERA_NAME    

with open( f"./{c.app_name}/deployment_overrides/override_camera.json", "w") as fd:
    json.dump(override_json, fd)

### Build app with container

In [ ]:
container_asset_name = 'code_asset'

In [ ]:
%%capture captured_output
# FIXME : without %%capture, browser tab crashes because of too much output from the command.

!cd ./people_counter_app && panorama-cli build \
    --container-asset-name {container_asset_name} \
    --package-path packages/{c.account_id}-{c.code_package_name}-1.0

In [ ]:
stdout_lines = captured_output.stdout.splitlines()
stderr_lines = captured_output.stderr.splitlines()
print("     :")
print("     :")
for line in stdout_lines[-30:] + stderr_lines[-30:]:
    print(line)

### Upload application to Panorama for deploying to devices

In [ ]:
!cd ./people_counter_app && panorama-cli package-application

### Ready for deploying to a device

Congrats! Your app is now ready to deploy to a device. Next, you can continue in this notebook to deploy the app programmatically or you can go to the Panorama console and deploying using the AWS Console. The console makes it easier to select camera streams and select the devices you want to deploy to. Programmatic deployment is faster to complete and easier to automate.

# Deploy app to device

Let's make sure the device we are deploying to is available.

In [ ]:
response = panorama_client.describe_device(
    DeviceId= DEVICE_ID
)

print('You are deploying to Device: {}'.format(response['Name']))

## Deploy app

You are ready to deploy your app. Below, you can see an example of how to use the AWS CLI to deploy the app. Alternatively, you can use the boto3 SDK as you did above for getting the device information.

In [ ]:
with open(f"./{c.app_name}/graphs/{c.app_name}/graph.json") as fd:
    manifest_payload = "'%s'" % json.dumps({"PayloadData":json.dumps(json.load(fd))})
    
with open(f"./{c.app_name}/deployment_overrides/override_camera.json") as fd:
    override_payload = "'%s'" % json.dumps({"PayloadData":json.dumps(json.load(fd))})

In [ ]:
res = !aws panorama create-application-instance \
    --name {c.app_name} \
    --default-runtime-context-device {DEVICE_ID} \
    --manifest-payload {manifest_payload} \
    --manifest-overrides-payload {override_payload}

res = ''.join(res)
print(res)
res_json = json.loads(res)

### Check Application Status

In [ ]:
app_id = res_json['ApplicationInstanceId']
print( "Application Instance Id :", app_id )

progress_dots = utils.ProgressDots()
while True:
    response = utils.describe_app(app_id)
    status = response['Status']
    progress_dots.update_status( f'{status} ({response["StatusDescription"]})' )
    if status in ('DEPLOYMENT_SUCCEEDED','DEPLOYMENT_FAILED'):
        break
    time.sleep(60)
    

# Clean up

In [ ]:
utils.remove_application( DEVICE_ID, app_id )